In [ ]:
# Import Vertex AI library
from google.cloud import aiplatform

In [ ]:
# Initialize connection
aiplatform.init(location='europe-west1')

#### Dataset preparation

In [ ]:
# Get dataset
dataset_id = 'your_dataset_id' # E.g.: projects/268076997885/locations/europe-west1/datasets/1312975213238943744
dataset = aiplatform.TabularDataset(dataset_id)

In [ ]:
# Display column names
dataset.column_names

In [ ]:
# Create column_specs object
column_specs = aiplatform.training_jobs.AutoMLTabularTrainingJob.get_auto_column_specs(
    dataset=dataset,
    target_column='stroke',
)

In [ ]:
# Display it
column_specs

In [ ]:
# Manually set transformation type
# One of: ['categorical', 'numeric', 'text', 'timestamp']
column_specs['gender'] = 'categorical'
column_specs['age'] = 'numeric'
column_specs

In [ ]:
# Exclude 'id' column from training
column_specs.pop('id', None)
column_specs

#### Train model

In [ ]:
# Name of the Model
display_name = 'stroke_classification_model'

# The type of prediction the Model is to produce.
# One of: ['classification', 'regression']
optimization_prediction_type = 'classification'

# Objective function the Model is to be optimized towards. 
# For classification (binary): 
# ['maximize-au-roc', 'minimize-log-loss', 'maximize-au-prc' 'maximize-precision-at-recall', 'maximize-recall-at-precision']
# For classification (multi class): 
# ['minimize-log-loss']
# For regression:
# ['minimize-rmse', 'minimize-mae', 'minimize-rmsle']
optimization_objective = 'minimize-log-loss'

In [ ]:
# Create training job
job = aiplatform.AutoMLTabularTrainingJob(
    display_name=display_name,
    optimization_prediction_type=optimization_prediction_type,
    optimization_objective=optimization_objective,
    column_specs=column_specs,
    location='europe-west1',
)

In [ ]:
# Run training job
model = job.run(
    dataset=dataset,
    target_column='stroke',
    training_fraction_split=0.7,
    validation_fraction_split=0.15,
    test_fraction_split=0.15,
    budget_milli_node_hours=1000
)

In [ ]:
# Retrieve model after training
model_id = 'your_model_id' # E.g.: projects/268076997885/locations/europe-west1/models/1312975213238942344
model = aiplatform.Model(model_id)

#### Export trained model

In [ ]:
# Check supported export formats
model.supported_export_formats

In [ ]:
# Export to GCS
model.export_model(export_format_id='tf-saved-model', artifact_destination='gs://c4ds-models')

#### Check model evaluation

In [ ]:
model.list_model_evaluations()

In [ ]:
evaluation_id = 'your_evaluation_id' # E.g.: 8061705251140478123
evaluation = model.get_model_evaluation(evaluation_id=evaluation_id)

In [ ]:
def get_model_evaluation_tabular_classification_sample(
    project: str,
    model_id: str,
    evaluation_id: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    To obtain evaluation_id run the following commands where LOCATION
    is the region where the model is stored, PROJECT is the project ID,
    and MODEL_ID is the ID of your model.

    model_client = aiplatform.gapic.ModelServiceClient(
        client_options={
            'api_endpoint':'LOCATION-aiplatform.googleapis.com'
            }
        )
    evaluations = model_client.list_model_evaluations(parent='projects/PROJECT/locations/LOCATION/models/MODEL_ID')
    print("evaluations:", evaluations)
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.ModelServiceClient(client_options=client_options)
    name = client.model_evaluation_path(
        project=project, location=location, model=model_id, evaluation=evaluation_id
    )
    response = client.get_model_evaluation(name=name)
    print("response:", response)


In [ ]:
project = '268076997885' # Project ID
model_id = '8895049068707840000' # The last part of model ID
location = 'europe-west1'
api_endpoint = 'europe-west1-aiplatform.googleapis.com'
get_model_evaluation_tabular_classification_sample(project=project,
                                                   model_id=model_id,
                                                   evaluation_id=evaluation_id,
                                                   location=location,
                                                   api_endpoint=api_endpoint)

#### Update model

In [ ]:
model.update(description='New model description')

#### Delete model

In [ ]:
model.delete()